In [2]:
from modules.bag_reader.bag_reader import READER
import cv2 
import numpy as np
import pandas as pd 

In [3]:
data = READER('/home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-24-00-07-50.bag')
data

[INFO]  Successfully created the data folder /home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-24-00-07-50.


In [4]:
data.read()


images saved


In [11]:
depth = np.load(data.depth_df.np_path[1])
print(depth)
print(depth.shape)

[[   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ... 1913. 1914. 1922.]
 [   0.    0.    0. ... 1913. 1913. 1912.]
 [   0.    0.    0. ... 1934. 1934. 1934.]]
(360, 640)


In [19]:
width = depth.shape[0]
length = depth.shape[1]
w_sections = np.linspace(0,width,5)
print(w_sections)
l_sections = np.linspace(0,length,5)
print(l_sections)

[  0.  90. 180. 270. 360.]
[  0. 160. 320. 480. 640.]


In [26]:
depth[:int(w_sections[1])-1,:int(l_sections[1])-1]

TypeError: slice indices must be integers or None or have an __index__ method